## Load the MFEM library

Any non-default libraries must be loaded before you can `#include` files that use them. We also need to tell cling where to find our mfem libraries (this is needed for Binder only, may be skipped locally).

In [ ]:
#pragma cling add_include_path("/home/jovyan/local/include")
#pragma cling add_library_path("/home/jovyan/local/lib")
#pragma cling load("libmfem")

## MFEM Example 1

This is the simplest MFEM example and a good starting point for new users. The example demonstrates the use of MFEM to define and solve an $H^1$ finite element discretization of the Laplace problem

$$
-\Delta u = 1
$$

with homogeneous Dirichlet boundary conditions $u=0$.

The example illustrates the use of the basic MFEM classes for defining the mesh, finite element space, as well as linear and bilinear forms corresponding to the left-hand side and right-hand side of the discrete linear system.

Compare with MFEM's [ex1.cpp](https://github.com/mfem/mfem/blob/master/examples/ex1.cpp) and PyMFEM's [ex1.py](https://github.com/mfem/PyMFEM/blob/master/examples/ex1.py).

In [ ]:
#include <fstream>
#include <iostream>
#include <sstream>

#include <mfem.hpp>
#include <glvis/xglvis.hpp>

In [ ]:
using namespace std;
using namespace mfem;

Mesh mesh = Mesh::MakeCartesian2D(5, 5, Element::TRIANGLE);
mesh.UniformRefinement();

H1_FECollection fec(2, mesh.Dimension());

FiniteElementSpace fespace(&mesh, &fec);
cout << "Number of finite element unknowns: " << fespace.GetTrueVSize() << endl;

Array<int> ess_tdof_list;
if (mesh.bdr_attributes.Size())
{
  Array<int> ess_bdr(mesh.bdr_attributes.Max());
  ess_bdr = 1;
  fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list);
}

LinearForm b(&fespace);
ConstantCoefficient one(1.0);
b.AddDomainIntegrator(new DomainLFIntegrator(one));
b.Assemble();

GridFunction x(&fespace);
x = 0.0;

BilinearForm a(&fespace);
a.AddDomainIntegrator(new DiffusionIntegrator(one));
a.Assemble();

OperatorPtr A;
Vector B, X;
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B);

cout << "Size of linear system: " << A->Height() << endl;

GSSmoother M((SparseMatrix&)(*A));
PCG(*A, M, B, X, 1, 200, 1e-12, 0.0);
a.RecoverFEMSolution(X, b, x);

## GLVis Visualization

For now we save the computational mesh and finite element solution in a string and pass that to the pyglvis widget, see https://github.com/GLVis/pyglvis.

In [ ]:
std::stringstream ss;
ss << "solution\n" << mesh << x << flush;

auto glv = glvis::glvis();
glv.show(ss.str() + "keys Rjml"); // the `+ "keys ...."' is optional
glv